<a href="https://colab.research.google.com/github/DavidABSiepmann/device_milk_adulteration/blob/master/Simula%C3%A7%C3%B5es_Distribui%C3%A7%C3%A3o_de_particulas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q miepython==2.3.0

import matplotlib.pyplot as plt
import numpy as np

from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error

path_drive = '/content/'

## Mie calculos

In [ ]:
import numpy as np
import miepython
import tqdm

def distribuition(
    meanRadius,
    stdDev,
    nRadius = 31,
    sphNumDensity = 1e8,
    volFraction = 0.1,
    VolOrConc = False):

    radArray = []
    funcArray = []
    numDensityArray = []
    totalSphereVolume = 0.0
    totalFuncSum = 0.0
    meanRadius *= 0.5
    maxRadius = 0

    if (nRadius <52):
        cutoffPercent = -0.0005*nRadius + 0.031 #from 2:52 change from  3% to 0.5%
    else:
        cutoffPercent = -0.00009*nRadius + 0.00968 #from 52:102 change from  0.5% to 0.05%
    sigma = stdDev

    mu = 0
    maxY = 1.0/(sigma * np.sqrt (2.0 * np.pi))
    minY = maxY*cutoffPercent
    dR = sigma/1e3
    
    ##forward search to find right end
    curR = mu + dR
    curY = (np.exp(-((curR-mu)*(curR-mu)/(2.0 * sigma * sigma))))/(sigma * np.sqrt (2.0 * np.pi))
    i = 2
    while (curY>minY):
        curR = mu + i*dR
        curY = (np.exp(-((curR-mu)*(curR-mu)/(2.0 * sigma * sigma))))/(sigma * np.sqrt (2.0 * np.pi))
        i += 1

    maxRadius = meanRadius + curR
    #Left end setting
    if (curR < meanRadius):
        minRadius = meanRadius - curR
    else:
        minRadius = 1e-8

    for i in range(0, nRadius):
        funcArray.append(1.0)
    stepR = (maxRadius - minRadius)/(nRadius -1)

    for i in range(0, nRadius):
        radArray.append(minRadius + i*stepR)
        temp = radArray[i]-meanRadius
        funcArray[i]  = (np.exp(-(temp*temp/(2.0 * stdDev * stdDev))))/(stdDev * np.sqrt (2.0 * np.pi))
        totalSphereVolume += funcArray[i] * 4.0 * np.pi * radArray[i] * radArray[i] * radArray[i] / 3.0
        totalFuncSum += funcArray[i]

    if (VolOrConc):
        factor = 1e9 * volFraction /totalSphereVolume;   #1mm3 x volume Fraction / Total volume of spheres
    else:
        factor = sphNumDensity/totalFuncSum

    for i in range(0, nRadius):
        numDensityArray.append(funcArray[i]*factor)

    radArray = np.array(radArray)
    numDensityArray = np.array(numDensityArray)

    return radArray, numDensityArray

def calc_scat(radArray, numDensityArray, theta, lambda_, crefraction, norm='4pi'):

    nRadius = len(numDensityArray)
    sumNumDen = sum(numDensityArray)

    m = crefraction
    xd = (2*np.pi*radArray)/lambda_
    mu = np.cos(theta/180*np.pi)

    buffer = np.zeros_like(miepython.i_unpolarized(m,xd[0],mu,norm))

    for i in range(0, nRadius):
        buffer += (miepython.i_unpolarized(m,xd[i],mu,norm)*numDensityArray[i])

    buffer /= sumNumDen

    return buffer

def gera_dataset(meanRadiusN, stdRadiusN, theta, lambda_, m, name, noise = 0):
    X = []
    y = []
    pbar = tqdm.tqdm(total=len(stdRadiusN)*len(meanRadiusN)+1)
    for stdRadius in stdRadiusN:
        for meanRadius in meanRadiusN:
            pbar.update(1)
            dist, distDensity = distribuition(meanRadius, stdRadius)
            
            distDensity += np.random.random(len(distDensity))*max(distDensity)*noise

            scat = calc_scat(dist, distDensity, theta, lambda_, m)
            scat += np.random.random(len(scat))*max(scat)*noise
            X.append(scat)
            d50 = meanRadius
            d25 = (meanRadius+2*stdRadius)*0.5
            y.append([d50, d25, stdRadius])

    X = np.array(X)
    y = np.array(y)

    dataset = np.concatenate((X, y), axis=1)
    np.savetxt(name, dataset, fmt="%10.8f" )
    pbar.update(1)
    pbar.close()
    return [X, y]

def loadDataset(name, MeanSigma = False, All = False):

    data = np.loadtxt(name)
    n = 3
    Xl = data[:,:-n]
    yl = data[:, len(data[0,:])-n::]

    if( All ):
        return [Xl,yl]

    if( MeanSigma ):
        yl = np.delete(yl, 1, 1)
        return [Xl,yl]
    else:
        return [Xl,yl[:,:-1]]

## Geração de dataset

In [ ]:
meanRadiusN = np.linspace(0.7,20,100)
stdRadiusN = np.linspace(0.05,1,100)
theta = np.linspace(-90,90,101)
lambda_ = 0.660
n_milk = 1.46
n_water = 1.33
coefRefracao = n_milk/n_water

meanRadiusN_test = np.random.random(20)*20+0.7
stdRadiusN_test =  np.random.random(10)*0.8+0.05

_ = gera_dataset(meanRadiusN, stdRadiusN, theta, lambda_, coefRefracao, f'{path_drive}/dataset-train-saidas.csv')
_ = gera_dataset(meanRadiusN_test, stdRadiusN_test, theta, lambda_, coefRefracao, f'{path_drive}/dataset-teste-saidas.csv')

## Carrega os datasets

In [ ]:
X, y = loadDataset(f'{path_drive}/dataset-train-saidas.csv')
X_t, y_t = loadDataset(f'{path_drive}/dataset-teste-saidas.csv')

print(X.shape, y.shape)

# Regresores

## Teste de diferentes alvos

In [ ]:
import pandas as pd

def test_models(X_train, y_train, X_test, y_test, name):
    models = [
        DecisionTreeRegressor(),
        LinearRegression(),
        KNeighborsRegressor(),
        Ridge(),
        Lasso(),
        ]

    lModels = []
    lR2 = []
    lMAE = []
    lRMSE = []

    for model in models:
        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

        lModels.append(model.__str__())
        lR2.append(np.mean(cross_val_score(model, X_train, y_train, scoring='r2', cv=cv, n_jobs=-1)))
        lMAE.append(abs(np.mean(cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1))))
        lRMSE.append(abs(np.mean(cross_val_score(model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1))))

    df = pd.DataFrame(
        {
            "Modelo": lModels, 
            "R2": lR2,
            "MAE": lMAE,
            "RMSE": lRMSE,
        }
    )

    df.to_csv(name, sep=';', index=False, float_format='%.4f')

    print(df)
    return df

In [ ]:
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print("\tPredict de media e desvio: ")
X, y = loadDataset('dataset2.csv', MeanSigma=True)
X_t, y_t = loadDataset('teste2.csv', MeanSigma=True)
test_models(X, y, X_t, y_t, "pred_mean_std.csv")

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print("\tPredict de d50 e d25: ")
X, y = loadDataset('dataset2.csv', MeanSigma=False)
X_t, y_t = loadDataset('teste2.csv', MeanSigma=False)
test_models(X, y, X_t, y_t, "pred_d50_d25.csv")

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print("\tPredict de d50, d25 e sigma: ")
X, y = loadDataset('dataset2.csv', All=True)
X_t, y_t = loadDataset('teste2.csv', All=True)
test_models(X, y, X_t, y_t, "pred_d50_d25_sigma.csv")


## Teste quantidade de angulos

In [ ]:
n_angulos = [3, 7, 11, 25, 101, 1000]


meanRadiusN = np.linspace(0.7,20,100)
stdRadiusN = np.linspace(0.05,1,100)
lambda_ = 0.660
n_milk = 1.46
n_water = 1.33
coefRefracao = n_milk/n_water

meanRadiusN_test = np.random.random(20)*20+1
stdRadiusN_test =  np.random.random(10)*0.8+0.05

In [ ]:
print("Gerando datasets")
for nAng in n_angulos:
    theta = np.linspace(-90,90,nAng)
    print("Angulos: ", theta)
    _ = gera_dataset(meanRadiusN, stdRadiusN, theta, lambda_, coefRefracao, f'{path_drive}/dataset_train_ang_{nAng}.csv')
    _ = gera_dataset(meanRadiusN_test, stdRadiusN_test, theta, lambda_, coefRefracao, f'{path_drive}/dataset_test_ang_{nAng}.csv')

In [ ]:
df = pd.DataFrame()

print("\tPredict de d50 e d25: ")
for nAng in n_angulos:
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print("Quantidade angulos: ", nAng)
    #print("Angulos: ", np.linspace(-180,180,nAng))
    X, y = loadDataset(f'{path_drive}/dataset_train_ang_{nAng}.csv', MeanSigma=False)
    X_t, y_t = loadDataset(f'{path_drive}/dataset_test_ang_{nAng}.csv', MeanSigma=False)
    df2 = test_models(X, y, X_t, y_t, f"qdte_ang_out_{nAng}.csv")
    df[f'R2-{nAng}'] = df2['R2']
    df[f'MAE-{nAng}'] = df2['MAE']
    df[f'RMSE-{nAng}'] = df2['RMSE']

df.to_csv(f'{path_drive}/resultado_tabela_angulos.csv', sep=';', index=False, float_format='%.4f')

### Teste Ruido

In [ ]:
n_noise = [0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.5]

meanRadiusN = np.linspace(0.7,20,60)
stdRadiusN = np.linspace(0.05,1,15)
lambda_ = 0.660
n_milk = 1.46
n_water = 1.33
coefRefracao = n_milk/n_water
theta = np.linspace(-180,180,101)

meanRadiusN_test = np.random.random(20)*20+1
stdRadiusN_test =  np.random.random(10)*0.8+0.05


In [ ]:
print("Gerando datasets")
for nNoise in n_noise:
    _ = gera_dataset(meanRadiusN, stdRadiusN, theta, lambda_, coefRefracao, f'{path_drive}/dataset_train_noise_{nNoise}_101ang.csv', noise=nNoise)
    _ = gera_dataset(meanRadiusN_test, stdRadiusN_test, theta, lambda_, coefRefracao, f'{path_drive}/dataset_test_noise_{nNoise}_101ang.csv', noise=nNoise)

In [ ]:
df = pd.DataFrame()
print("\tPredict de d50 e d25: ")
for nNoise in n_noise:
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print("Noise: ", nNoise)
    X, y = loadDataset(f'{path_drive}/dataset_train_noise_{nNoise}_101ang.csv', MeanSigma=False)
    X_t, y_t = loadDataset(f'{path_drive}/dataset_test_noise_{nNoise}_101ang.csv', MeanSigma=False)
    df2 = test_models(X, y, X_t, y_t, "temp.csv")
    df[f'R2-{nNoise}'] = df2['R2']
    df[f'MAE-{nNoise}'] = df2['MAE']
    df[f'RMSE-{nNoise}'] = df2['RMSE']
df
df.to_csv(f'{path_drive}/resultado_tabela_noise_101ang.csv', sep=';', index=False, float_format='%.4f')

# Visualização do espalhamento

In [ ]:
n_angulos = [3, 7, 11, 25, 101, 1000]
xlb = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

plt.rcParams.update({'font.size': 12})

meanRadius = 7.4
stdRadius = 0.5
lambda_ = 0.660
n_milk = 1.46
n_water = 1.33
coefRefracao = n_milk/n_water

dist, distDensity = distribuition(meanRadius, stdRadius)

plt.figure(figsize=(5, 5))
plt.plot(dist*2, distDensity)
plt.xlabel('Tamanho da partícula [mícrons]', fontsize=12)
plt.ylabel('Concentração [Esferas em 1 mm³]', fontsize=12)
plt.savefig("distribuicao_qtde_ang.png", dpi=300)

fig, axes = plt.subplots(len(n_angulos)//2, 2, sharex=True, sharey=True, figsize=(7, 6))

i = 0
for a in range(len(n_angulos)//2):
    for b in [0, 1]:
        theta = np.linspace(-90,90, n_angulos[i])
        scat = calc_scat(dist, distDensity, theta, lambda_, coefRefracao)
        axes[a,b].plot(theta, scat)
        axes[a,b].set_title(xlb[i], fontsize=12)
        i += 1

fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel("Ângulo [graus]", fontsize=12)
plt.ylabel("Luz espalhada não polarizada [1/sr]", fontsize=12)

plt.savefig(f"luz_espalhada_ang.png", dpi=600)

In [ ]:
n_noise = [0.0050, 0.010, 0.050, 0.10, 0.20, 0.50]
xlb = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

plt.rcParams.update({'font.size': 12})

meanRadius = 7.4
stdRadius = 0.5
lambda_ = 0.660
n_milk = 1.46
n_water = 1.33
coefRefracao = n_milk/n_water
theta = np.linspace(-90,90, 101)

dist, distDensity = distribuition(meanRadius, stdRadius)
fig, axes = plt.subplots(len(n_noise)//2, 2, sharex=True, sharey=True, figsize=(7, 6))

i = 0
for a in range(len(n_noise)//2):
    for b in [0, 1]:
        scat = calc_scat(dist, distDensity, theta, lambda_, coefRefracao)
        #axes[a,b].plot(theta, scat, 'k-')
        scat += np.random.random(len(scat))*max(scat)*n_noise[i]
        axes[a,b].plot(theta, scat, 'b.-', markersize=3)
        axes[a,b].set_title(xlb[i], fontsize=12)
        i += 1

fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel("Ângulo [graus]", fontsize=12)
plt.ylabel("Luz espalhada não polarizada [1/sr]", fontsize=12)

plt.savefig(f"luz_espalhada_ruido.png", dpi=600)